<a href="https://colab.research.google.com/github/EpicMike87/AIProject/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gym-anytrading gym sb3_contrib finta

# Gym stuff
import gymnasium as gym
import gym_anytrading

# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C
from sb3_contrib import RecurrentPPO

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from gym_anytrading.envs import StocksEnv
from finta import TA

from google.colab import drive
drive.mount('/content/drive')
%ls
%cd drive
%ls
%cd My \Drive
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive/  sample_data/
/content/drive
MyDrive/
/content/drive/My Drive
 20180629_155959.jpg
 549271.pdf
 551375_eticket.pdf
 a5leaflet_CS.pdf
'Alternative Dates Form 20122013.docx.gdoc'
'Application Form November 2011 (2).doc'
'Application Pack.doc'
'Application - Sean Laughlin.gdoc'
 Application-SeanLaughlin.gdoc
'Bank Details Form.gdoc'
 christophersingleton.doc
'Colab Notebooks'/
'Components (1).docx.gdoc'
 Components.docx.gdoc
'Contract of employment Sean Laughlin.doc'
'Copy of Copy of Wonderland Staking and Minting Calculator (4,4).gsheet'
'Copy of Year of Industry draft 2012.doc.gdoc'
 coverphoto2.png
 coverphoto.png
'CV Grameen January 2016 (1).gdoc'
'CV Grameen January 2016 (2).gdoc'
'CV Grameen January 2016 (3).gdoc'
'CV Grameen January 2016 (4).gdoc'
'CV Grameen January 2016.doc'
'CV Grameen January 2016.gdoc'
 C.V.rtf
 cv_seanlaughlin.doc
 DAN175991.

In [12]:
df = pd.read_json('data/updated_spy_data.json')

df['Date'] = pd.to_datetime(df['timestamp'])
df.rename(columns={'close': 'Close'}, inplace=True)
df.rename(columns={'open': 'Open'}, inplace=True)
df.rename(columns={'low': 'Low'}, inplace=True)
df.rename(columns={'high': 'High'}, inplace=True)
df.rename(columns={'volume': 'Volume'}, inplace=True)

df.drop(columns=['timestamp'], inplace=True)
df.drop(columns=['stochastic_k'], inplace=True)
df.drop(columns=['stochastic_d'], inplace=True)
df.drop(columns=['macd_line'], inplace=True)
df.drop(columns=['signal_line'], inplace=True)
df.drop(columns=['macd_histogram'], inplace=True)
df.drop(columns=['momentum'], inplace=True)

macd_result = TA.MACD(df, 12, 26)
df['MACD'] = macd_result['MACD']
df['MACD_SIGNAL'] = macd_result['SIGNAL']
df['MOM'] = TA.MOM(df, 10)
df.fillna(0, inplace=True)

In [13]:
df = df.drop(df.index[:100])
df.head(15)

,Volume,High,Low,Open,Close,Date,MACD,MACD_SIGNAL,MOM
100,490816.0,450.019989,449.592194,449.734985,450.000000,2023-11-15 18:30:00,-0.170777,-0.181059,0.041412
101,465766.0,450.059998,449.850006,449.989990,449.959991,2023-11-15 18:35:00,-0.137176,-0.172283,-0.145020
102,501155.0,450.190002,449.945007,449.950012,450.070007,2023-11-15 18:40:00,-0.100514,-0.157929,0.090118
103,251484.0,450.154999,449.970001,450.089996,450.089996,2023-11-15 18:45:00,-0.069051,-0.140153,0.320007
104,474674.0,450.309998,450.049988,450.160004,450.170013,2023-11-15 18:50:00,-0.037233,-0.119569,0.620026
105,990657.0,450.260010,450.070007,450.170013,450.225006,2023-11-15 18:55:00,-0.007494,-0.097154,1.115021
106,836886.0,450.475006,450.204987,450.230011,450.359985,2023-11-15 19:00:00,0.026657,-0.072392,1.349976
107,475565.0,450.470001,450.279999,450.359985,450.339996,2023-11-15 19:05:00,0.051514,-0.047611,1.148010
108,559296.0,450.434998,450.279999,450.350006,450.295013,2023-11-15 19:10:00,0.066815,-0.024726,0.664917
109,842069.0,450.309998,449.790009,450.295013,449.919098,2023-11-15 19:15:00,0.048060,-0.010169,0.180084


In [14]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Low', 'Volume','MACD', 'MACD_SIGNAL', 'MOM']].to_numpy()[start:end]
    return prices, signal_features

In [16]:
class MyCustomEnv(StocksEnv):
    _process_data = add_signals

env2 = MyCustomEnv(df=df, window_size=12, frame_bound=(80,500))

In [17]:
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Volume,High,Low,Open,Close,Date,MACD,MACD_SIGNAL,MOM
100,490816.0,450.019989,449.592194,449.734985,450.000000,2023-11-15 18:30:00,-0.170777,-0.181059,0.041412
101,465766.0,450.059998,449.850006,449.989990,449.959991,2023-11-15 18:35:00,-0.137176,-0.172283,-0.145020
102,501155.0,450.190002,449.945007,449.950012,450.070007,2023-11-15 18:40:00,-0.100514,-0.157929,0.090118
103,251484.0,450.154999,449.970001,450.089996,450.089996,2023-11-15 18:45:00,-0.069051,-0.140153,0.320007
104,474674.0,450.309998,450.049988,450.160004,450.170013,2023-11-15 18:50:00,-0.037233,-0.119569,0.620026


In [18]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
model = RecurrentPPO("MlpLstmPolicy", env, verbose=1)
model.learn(total_timesteps=1000000)

#stop cell execution once explained variance is consistently high (near 1)

Using cpu device
----------------------------
| time/              |     |
|    fps             | 374 |
|    iterations      | 1   |
|    time_elapsed    | 0   |
|    total_timesteps | 128 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 38           |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 256          |
| train/                  |              |
|    approx_kl            | 0.0001726239 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.693       |
|    explained_variance   | -0.915       |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0792       |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00313     |
|    value_loss           | 0.232        |
------------------------------------------
------

In [ ]:
env = MyCustomEnv(df=df, window_size=12, frame_bound=(2500,3000))
obs = env.reset()
obs = obs[0]
prev_action = None
data_to_plot = []
while True:
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, reward, terminated, truncated, info = env.step(action)


    done = truncated or terminated

    print("info", info)
    if done:
        print("info", info)
        break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all(data_to_plot)
plt.show()